In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import fiona
import glob

In [2]:
data_interim_dir = "../../data/interim/"

# Read network

In [5]:
node_file = data_interim_dir + "step5_tidy_roadway/" + "node.geojson"
node_gdf = gpd.read_file(node_file)

shape_gdf = gpd.read_file(data_interim_dir + "step5_tidy_roadway/" 
                          + "shape.geojson")

DriverError: Failed to read GeoJSON data

In [3]:
link_file = data_interim_dir + "step5_tidy_roadway/" + "link.feather"

link_df = pd.read_feather(link_file)

In [ ]:
shape_gdf.columns

In [ ]:
link_gdf = pd.merge(link_df,
                    shape_gdf[["id", "geometry"]],
                   how = "left",
                   left_on = "shstGeometryId",
                   right_on = "id")

link_gdf = gpd.GeoDataFrame(link_gdf,
                            crs="EPSG:4326")

In [ ]:
link_gdf.info()

# Review tomtom conflation

In [13]:
# loading tomtom data source

gdb_file = "../../data/external/tomtom/Network_region.gdb"

# Get all the layers from the .gdb file 

layers = fiona.listlayers(gdb_file)

# street link layer

tomtom_raw_gdf = gpd.read_file(gdb_file, layer = 'mn_nw_BayArea')

# convert to ESPG lat-lon

tomtom_raw_gdf = tomtom_raw_gdf.to_crs({'init': 'epsg:4326'})
print(tomtom_raw_gdf.crs)

# generating unique handle for tomtom

tomtom_raw_gdf["tomtom_link_id"] = range(1, len(tomtom_raw_gdf)+1)

{'init': 'epsg:4326'}


In [25]:
tomtom_raw_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 602060 entries, 0 to 602059
Data columns (total 54 columns):
ID                602060 non-null float64
FEATTYP           602060 non-null int64
FT                602060 non-null int64
F_JNCTID          602060 non-null float64
F_JNCTTYP         602060 non-null int64
T_JNCTID          602060 non-null float64
T_JNCTTYP         602060 non-null int64
PJ                602060 non-null int64
METERS            602060 non-null float64
FRC               602060 non-null int64
NETCLASS          602060 non-null int64
NETBCLASS         602060 non-null int64
NET2CLASS         602060 non-null int64
NAME              602060 non-null object
NAMELC            602060 non-null object
SOL               602060 non-null int64
NAMETYP           602060 non-null int64
CHARGE            602060 non-null object
SHIELDNUM         602060 non-null object
RTETYP            602060 non-null int64
RTEDIR            602060 non-null object
RTEDIRVD          602060 non

In [23]:
# read tomtom unmatched

shst_unmatch_gdf = pd.DataFrame()

shst_file = glob.glob(data_interim_dir + "tomtom/ped_rules/**/*.unmatched.geojson", recursive = True)

print("----------start reading shst extraction data-------------")
for i in shst_file:
    print("reading shst extraction data : ", i)
    new = gpd.read_file(i)
    new['source'] = i
    shst_unmatch_gdf = pd.concat([shst_unmatch_gdf, new],
                        ignore_index = True,
                        sort = False)
print("----------finished reading shst extraction data-------------")

----------start reading shst extraction data-------------
reading shst extraction data :  D:/MTC/data/interim/tomtom/ped_rules\10_tomtom.out.unmatched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom/ped_rules\11_tomtom.out.unmatched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom/ped_rules\12_tomtom.out.unmatched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom/ped_rules\13_tomtom.out.unmatched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom/ped_rules\14_tomtom.out.unmatched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom/ped_rules\1_tomtom.out.unmatched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom/ped_rules\2_tomtom.out.unmatched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom/ped_rules\3_tomtom.out.unmatched.geojson
reading shst extraction data :  D:/MTC/data/interim/tomtom/ped_rules\4_tomtom.out.unmatched.geojson
reading shst extraction data :  D:/MT

In [24]:
shst_unmatch_gdf

,tomtom_link_id,geometry,source
0,4339,LINESTRING (-121.9982259999756 37.749368000002...,D:/MTC/data/interim/tomtom/ped_rules\10_tomtom...
1,7009,LINESTRING (-122.0849662997308 38.003465899507...,D:/MTC/data/interim/tomtom/ped_rules\10_tomtom...
2,7569,LINESTRING (-122.0862539996563 37.925149999630...,D:/MTC/data/interim/tomtom/ped_rules\10_tomtom...
3,9372,LINESTRING (-122.2121910004465 37.885682999960...,D:/MTC/data/interim/tomtom/ped_rules\10_tomtom...
4,12513,LINESTRING (-121.9387150001248 38.015353999737...,D:/MTC/data/interim/tomtom/ped_rules\10_tomtom...
5,14000,LINESTRING (-122.2793110002485 37.961569999993...,D:/MTC/data/interim/tomtom/ped_rules\10_tomtom...
6,14158,LINESTRING (-122.2096409995619 38.054923999709...,D:/MTC/data/interim/tomtom/ped_rules\10_tomtom...
7,14579,LINESTRING (-122.0809419999716 37.831067000145...,D:/MTC/data/interim/tomtom/ped_rules\10_tomtom...
8,15827,LINESTRING (-122.2534680002536 37.893244000122...,D:/MTC/data/interim/tomtom/ped_rules\10_tomtom...
9,16147,LINESTRING (-121.9858669998563 37.847041000588...,D:/MTC/data/interim/tomtom/ped_rules\10_tomtom...


In [14]:
link_gdf[link_gdf.tomtom_unique_id.isnull()]

,FRC,F_JNCTID,LANES,T_JNCTID,access,area,bike_access,bridge,drive_access,est_width,...,tomtom_ID,tomtom_unique_id,tunnel,u,v,walk_access,wayId,width,id,geometry
4,NaN,NaN,NaN,NaN,nan,nan,1,nan,1,nan,...,NaN,NaN,nan,4545575571,4545575563,1,458575920,nan,550ef74674d67256659c5ab80d35d7a7,"LINESTRING (-122.0118628 37.3784474, -122.0107..."
6,NaN,NaN,NaN,NaN,nan,nan,1,nan,1,nan,...,NaN,NaN,nan,4305402275,4305402282,1,431332715,nan,0a1b0305243b6d7658604b2fa1026086,"LINESTRING (-122.0877569 37.6706708, -122.0876..."
7,NaN,NaN,NaN,NaN,nan,nan,1,nan,0,nan,...,NaN,NaN,nan,5021128861,65381081,1,8923666,nan,fd8889120fd8b59c2e046dc8a34da4d2,"LINESTRING (-121.3351831 37.01963780000001, -1..."
8,NaN,NaN,NaN,NaN,nan,nan,1,nan,1,nan,...,NaN,NaN,nan,4925258551,4332522355,1,501844002,nan,08226050f43562c1d05cf9e36255507f,"LINESTRING (-122.0267767 37.930987, -122.02660..."
12,NaN,NaN,NaN,NaN,nan,nan,0,nan,0,nan,...,NaN,NaN,nan,5894779601,5536817173,1,577595380,nan,27957b82bf090e32fc79be43af3458b1,"LINESTRING (-121.9320685 37.3160261, -121.9320..."
22,NaN,NaN,NaN,NaN,nan,nan,1,nan,0,nan,...,NaN,NaN,nan,1473679962,1473679945,1,133930995,nan,5e4fe4d33fa842a6a3c91b45774b1e09,"LINESTRING (-122.2402939 37.4900318, -122.2400..."
23,NaN,NaN,NaN,NaN,nan,nan,1,nan,1,nan,...,NaN,NaN,nan,4932505529,4932505524,1,502902373,nan,f06d238bd8c5797d49ccd956960bbff1,"LINESTRING (-121.8258008 38.0116667, -121.8257..."
27,NaN,NaN,NaN,NaN,nan,nan,1,nan,1,nan,...,NaN,NaN,nan,4923736926,4923736941,1,501614839,nan,1429235fffef0c4a79ed17dbd37019a8,"LINESTRING (-122.7547949 38.4230409, -122.7550..."
29,NaN,NaN,NaN,NaN,nan,nan,1,nan,0,nan,...,NaN,NaN,nan,2088694605,2088694590,1,198773774,nan,29faa94f7bbbeb7830cedd5c73bc12a4,"LINESTRING (-121.7906751 37.2690919, -121.7909..."
31,NaN,NaN,NaN,NaN,nan,nan,1,nan,1,nan,...,NaN,NaN,nan,3373432239,3373432236,1,330379888,nan,cc4c713536342b575b3747ab9affc7aa,"LINESTRING (-122.5409339 38.004366, -122.54080..."


In [26]:
# export unmatched tomtom shape

tomtom_raw_gdf[(tomtom_raw_gdf.tomtom_link_id.isin(
    shst_unmatch_gdf.tomtom_link_id.tolist()))][["ID", "tomtom_link_id", "geometry",
                                                "FRC", "LANES"]].to_file(
data_interim_dir + "unmatched_tomtom.geojson", driver = "GeoJSON")

In [27]:
# export shst shapes without tomtom match

link_gdf[link_gdf.tomtom_unique_id.isnull()][["shstReferenceId",
                                              "shstGeometryId", "geometry"]].to_file(
data_interim_dir + "link_not_conflated_with_tomtom.geojson", driver = "GeoJSON")

In [33]:
tomtom_raw_gdf.FRC.value_counts()

7    210545
6    175574
4     83670
5     64720
8     28729
3     20513
1      9304
0      7707
2      1298
Name: FRC, dtype: int64

In [6]:
# tomtom FRC dictionary

tomtom_FRC_dict = {
    0: "0-Motorway, Freeway, or Other Major Road",
    1: "1-Major Road Less Important than a Motorway",
    2: "2-Other Major Road",
    3: "3-Secondary Road",
    4: "4-Local Connecting Road",
    5: "5-Local Road of High Importance",
    6: "6-Local Road",
    7: "7-Local Road of Minor Importance",
    8: "8-Other Road"
}

link_df["tomtom_FRC_def"] = link_df["tomtom_FRC"].map(tomtom_FRC_dict)

In [7]:
link_df.tomtom_FRC_def.value_counts()

6-Local Road                                   345262
7-Local Road of Minor Importance               325150
5-Local Road of High Importance                 83747
4-Local Connecting Road                         71881
8-Other Road                                    35400
3-Secondary Road                                14483
1-Major Road Less Important than a Motorway      1851
0-Motorway, Freeway, or Other Major Road         1217
2-Other Major Road                                408
Name: tomtom_FRC_def, dtype: int64

In [8]:
link_df.TM2_FT.value_counts()

4.0    540491
7.0    141340
0.0     56409
3.0     13888
2.0      3271
5.0      2168
1.0       156
Name: TM2_FT, dtype: int64

In [9]:
link_df.TM2Marin_FT.value_counts()

4.0     517477
7.0     137608
0.0      56409
14.0     19369
3.0      13393
12.0      5007
2.0       3269
11.0      2642
5.0       2189
13.0       202
1.0        155
Name: TM2Marin_FT, dtype: int64

In [10]:
link_df[link_df.drive_access==1].TM2Marin_FT.value_counts()

4.0     418486
7.0     136628
3.0      13240
14.0     12986
0.0       7898
12.0      4852
2.0       3216
11.0      2613
5.0       2172
13.0       194
1.0        155
Name: TM2Marin_FT, dtype: int64

In [11]:
TM2_FT_dict = {
    0: "0-Connector",
    1: "1-Freeway to Freeway",
    2: "2-Freeway",
    3: "3-Expressway",
    4: "4-Collector",
    5: "5-Ramp",
    6: "6-Special Facility",
    7: "7-Major Arterial",
}

link_df["TM2_FT_def"] = link_df["TM2_FT"].map(TM2_FT_dict)

link_df.TM2_FT_def.value_counts()

4-Collector             540491
7-Major Arterial        141340
0-Connector              56409
3-Expressway             13888
2-Freeway                 3271
5-Ramp                    2168
1-Freeway to Freeway       156
Name: TM2_FT_def, dtype: int64

In [12]:
link_df.columns

Index(['access', 'area', 'bike_access', 'bridge', 'drive_access', 'est_width',
       'fromIntersectionId', 'highway', 'id', 'junction', 'key', 'landuse',
       'lanes', 'link', 'maxspeed', 'name', 'oneWay', 'ref', 'roadway',
       'roundabout', 'service', 'shstGeometryId', 'shstReferenceId',
       'toIntersectionId', 'tunnel', 'u', 'v', 'walk_access', 'wayId', 'width',
       'tomtom_unique_id', 'tomtom_ID', 'F_JNCTID', 'T_JNCTID', 'tomtom_lanes',
       'tomtom_FRC', 'tomtom_name', 'tomtom_shieldnum', 'tomtom_rtedir',
       'TM2_A', 'TM2_B', 'TM2_LANES', 'TM2_FT', 'TM2_ASSIGNABLE', 'TM2Marin_A',
       'TM2Marin_B', 'TM2Marin_LANES', 'TM2Marin_FT', 'TM2Marin_ASSIGNABLE',
       'sfcta_A', 'sfcta_B', 'sfcta_FT', 'sfcta_STREETNAME', 'sfcta_LANE_AM',
       'sfcta_LANE_OP', 'sfcta_LANE_PM', 'pems_lanes_FF', 'pems_lanes_FR',
       'pems_lanes_HV', 'pems_lanes_ML', 'pems_lanes_OR', 'length',
       'tomtom_FRC_def', 'TM2_FT_def'],
      dtype='object')

In [13]:
# write tableau data

tableau_fields = ["shstReferenceId", "roadway", "lanes", "drive_access", "bike_access", "walk_access",
                 "tomtom_FRC", "tomtom_FRC_def", "tomtom_lanes", "tomtom_unique_id", 
                  'tomtom_name', 'tomtom_shieldnum', 'tomtom_rtedir', 'TM2Marin_A',
                   'TM2Marin_B', 'TM2Marin_FT', 'TM2Marin_LANES', 'TM2Marin_ASSIGNABLE', 
                  'TM2_A', 'TM2_B', 'TM2_FT', "TM2_FT_def", 'TM2_LANES', 'TM2_ASSIGNABLE', 
                  "sfcta_A", 'sfcta_B', "sfcta_STREETNAME", 'sfcta_FT', 'sfcta_LANE_AM', 'sfcta_LANE_OP',
                   'sfcta_LANE_PM', 'pems_lanes_FF', 'pems_lanes_FR',
       'pems_lanes_HV', 'pems_lanes_ML', 'pems_lanes_OR']

link_df[tableau_fields].rename(columns = {"lanes" : "lanes_osm"}
                              ).to_csv(data_interim_dir + "conflation_review_for_tableau.csv",
                              index = False)